Model Validation

Qu’est-ce que la validation de modèle ?
Vous voudrez évaluer presque tous les modèles que vous construisez. Dans la plupart des cas (mais pas tous), la mesure pertinente de la qualité d’un modèle est la précision prédictive. En d’autres termes, est-ce que les prédictions du modèle seront proches de ce qui se passe réellement.

Beaucoup de gens commettent une grosse erreur lorsqu’ils mesurent la précision prédictive. Ils font des prédictions sur leurs données d’entraînement et comparent ces prédictions aux valeurs cibles de ces mêmes données d’entraînement. Vous verrez le problème avec cette méthode et comment le résoudre dans un instant, mais réfléchissons d’abord à comment procéder.

Résumer la qualité du modèle
Il faut d’abord résumer la qualité du modèle de manière compréhensible. Si vous comparez les valeurs prédites et réelles pour 10 000 maisons, vous trouverez sûrement un mélange de bonnes et mauvaises prédictions. Passer en revue une liste de 10 000 valeurs prédites et réelles serait inutile. Il faut donc résumer cela en une seule métrique.

La métrique Mean Absolute Error (MAE)
Il existe de nombreuses métriques pour résumer la qualité d’un modèle, mais nous allons commencer avec une appelée Mean Absolute Error (MAE), ou erreur moyenne absolue.

Décomposons cette métrique en commençant par le dernier mot : erreur.

L’erreur de prédiction pour chaque maison est : erreur = valeur réelle − valeur prédite; 

Par exemple, si une maison coûte 150 000 $ et que vous avez prédit 100 000 $, l’erreur est de 50 000 $.

Avec la métrique MAE, on prend la valeur absolue de chaque erreur, ce qui convertit chaque erreur en un nombre positif. Ensuite, on calcule la moyenne de ces erreurs absolues. C’est notre mesure de la qualité du modèle.

En termes simples, cela signifie : En moyenne, nos prédictions s’écartent d’environ X.

In [3]:
# Data Loading Code Hidden Here
import pandas as pd

# Load data
melbourne_file_path = 'melb_data.csv'
melbourne_data = pd.read_csv(melbourne_file_path) 
# Filter rows with missing price values
filtered_melbourne_data = melbourne_data.dropna(axis=0)
# Choose target and features
y = filtered_melbourne_data.Price
melbourne_features = ['Rooms', 'Bathroom', 'Landsize', 'BuildingArea', 
                        'YearBuilt', 'Lattitude', 'Longtitude']
X = filtered_melbourne_data[melbourne_features]

from sklearn.tree import DecisionTreeRegressor
# Define model
melbourne_model = DecisionTreeRegressor()
# Fit model
melbourne_model.fit(X, y)

,criterion,'squared_error'
,splitter,'best'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,None
,random_state,None
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,ccp_alpha,0.0


 a model, here is how we calculate the mean absolute error:

In [4]:
from sklearn.metrics import mean_absolute_error

predicted_home_prices = melbourne_model.predict(X)
mean_absolute_error(y, predicted_home_prices)

434.71594577146544

### Le problème des scores « in-sample » (sur l’échantillon d’entraînement)
La mesure que nous venons de calculer peut être appelée un score in-sample.
Nous avons utilisé un seul et même « échantillon » de maisons à la fois pour construire le modèle et pour l’évaluer. 

Voici pourquoi ce n’est pas une bonne idée : 
Imaginez que, sur l’ensemble du marché immobilier, la couleur de la porte n’a aucun lien avec le prix d’une maison.
Mais, dans l’échantillon de données que vous avez utilisé pour entraîner votre modèle, toutes les maisons avec une porte verte étaient très chères.
Le rôle du modèle est de détecter des motifs qui permettent de prédire les prix. Il va donc remarquer ce lien (erroné) et prédira systématiquement des prix élevés pour les maisons avec une porte verte. Puisque ce motif a été appris à partir des données d'entraînement, le modèle semblera précis sur ces mêmes données.Mais si ce motif ne se vérifie pas dans de nouvelles données, alors le modèle sera très inexact en pratique.

### Pourquoi utiliser des données de validation ?
La valeur réelle d’un modèle vient de sa capacité à faire des prédictions sur des données nouvelles (jamais vues).
C’est pourquoi on mesure la performance sur des données qui n’ont pas servi à l’entraînement du modèle.
La méthode la plus simple pour cela consiste à exclure une partie des données lors de l’entraînement, puis à les utiliser pour tester la précision du modèle. Ces données sont appelées données de validation.

## Codage : scinder les données
La bibliothèque scikit-learn fournit une fonction appelée train_test_split qui permet de diviser les données en deux parties.
Une partie sera utilisée comme données d'entraînement pour ajuster le modèle.
L’autre partie sera utilisée comme données de validation pour calculer l’erreur moyenne absolue (MAE).

In [5]:
from sklearn.model_selection import train_test_split

# split data into training and validation data, for both features and target
# The split is based on a random number generator. Supplying a numeric value to
# the random_state argument guarantees we get the same split every time we
# run this script.
train_X, val_X, train_y, val_y = train_test_split(X, y, random_state = 0)
# Define model
melbourne_model = DecisionTreeRegressor()
# Fit model
melbourne_model.fit(train_X, train_y)

# get predicted prices on validation data
val_predictions = melbourne_model.predict(val_X)
print(mean_absolute_error(val_y, val_predictions))

263973.2104583602


Notre erreur moyenne absolue (MAE) sur les données d'entraînement (in-sample) était d’environ 500 dollars.
Mais sur les données de validation (out-of-sample), elle dépasse les 250 000 dollars.

C’est la différence entre un modèle presque parfaitement juste…
et un modèle inutilisable dans la plupart des cas concrets.

À titre de comparaison, la valeur moyenne des maisons dans les données de validation est de 1,1 million de dollars.
Donc, l’erreur sur les nouvelles données représente environ un quart de la valeur moyenne d’une maison.

Il existe de nombreuses façons d'améliorer ce modèle, par exemple :
en expérimentant avec de meilleures variables explicatives (features), ou en testant d'autres types de modèles.